In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_breast_cancer

In [7]:
data = load_breast_cancer()
data.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [10]:
print(data.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry 
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 3 is Mean Radius, f

In [15]:
X = pd.DataFrame(data.data)
X.columns= data.feature_names
y = data.target

X.shape, y.shape

((569, 30), (569,))

In [17]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.2, random_state= 2)
X_train.shape, X_test.shape

((455, 30), (114, 30))

## Feature selection by feature importance of random classifier

In [81]:
# calculates importance of all features and also the mean of all importance and selects features whose importance is greater than mean
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100, random_state = 0 , n_jobs = -1))
sel.fit(X_train, y_train)
sel.get_support()

array([False, False,  True,  True, False, False,  True,  True, False,
       False, False, False, False,  True, False, False, False, False,
       False, False,  True, False,  True,  True, False, False, False,
        True, False, False])

In [97]:
X_train.columns
len(X_train.columns)


30

In [91]:
features = X_train.columns[sel.get_support()]
features
len(features)

9

In [84]:
np.mean(sel.estimator_.feature_importances_)

0.03333333333333333

In [85]:
sel.estimator_.feature_importances_

array([0.02600613, 0.01414504, 0.08644319, 0.04469739, 0.00646959,
       0.00423233, 0.05055472, 0.08076548, 0.0026119 , 0.00432518,
       0.02235071, 0.0053831 , 0.01472101, 0.04473355, 0.00313867,
       0.00278753, 0.00482326, 0.00325324, 0.0027565 , 0.00296122,
       0.11556779, 0.01367086, 0.17457203, 0.09932068, 0.01230355,
       0.01505478, 0.02127351, 0.10357992, 0.00542455, 0.0120726 ])

In [86]:
X_train_rfc = sel.transform(X_train)
X_test_rfc = sel.transform(X_test)

In [93]:
 def run_randomForest(X_train,X_test,y_train,y_test):
    clf = RandomForestClassifier(n_estimators = 100, random_state = 0, n_jobs = -1)
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    print('Accuracy: ', accuracy_score(y_test,y_pred))

In [94]:
%%time
run_randomForest(X_train_rfc,X_test_rfc,y_train,y_test)

Accuracy:  0.9473684210526315
Wall time: 414 ms


In [99]:
%%time
run_randomForest(X_train,X_test,y_train,y_test)
len(X_train.columns)

Accuracy:  0.9385964912280702
Wall time: 406 ms


30

## Recursive Feature Elimination(RFE)

In [50]:
from sklearn.feature_selection import RFE
sel = RFE(RandomForestClassifier(n_estimators = 100, n_jobs = -1 , random_state = 0),n_features_to_select = 15)
sel.fit(X_train,y_train)

RFE(estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                     criterion='gini', max_depth=None,
                                     max_features='auto', max_leaf_nodes=None,
                                     min_impurity_decrease=0.0,
                                     min_impurity_split=None,
                                     min_samples_leaf=1, min_samples_split=2,
                                     min_weight_fraction_leaf=0.0,
                                     n_estimators=100, n_jobs=-1,
                                     oob_score=False, random_state=0, verbose=0,
                                     warm_start=False),
    n_features_to_select=15, step=1, verbose=0)

In [52]:
sel.get_support()

array([ True,  True,  True,  True, False, False,  True,  True, False,
       False, False, False, False,  True, False, False, False, False,
       False, False,  True,  True,  True,  True,  True,  True,  True,
        True, False, False])

In [55]:
features = X_train.columns[sel.get_support()]
features

Index(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean concavity', 'mean concave points', 'area error', 'worst radius',
       'worst texture', 'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points'],
      dtype='object')

In [57]:
X_train_rfe = sel.transform(X_train)
X_test_rfe = sel.transform(X_test)

In [113]:
clf = RandomForestClassifier(n_estimators = 100, random_state = 0 , n_jobs = -1)
clf.fit(X_train_rfe,y_train)
y_pred = clf.predict(X_test_rfe)
print(accuracy_score(y_test,y_pred))


0.9385964912280702


In [114]:
for index in range(1, 31):
    sel = RFE(RandomForestClassifier(n_estimators=100, random_state=0), n_features_to_select = index)
    sel.fit(X_train, y_train)
    X_train_rfe = sel.transform(X_train)
    X_test_rfe = sel.transform(X_test)
    print('Selected Feature: ', index)
    run_randomForest(X_train_rfe, X_test_rfe, y_train, y_test)
    print()

Selected Feature:  1
Accuracy:  0.868421052631579

Selected Feature:  2
Accuracy:  0.8859649122807017

Selected Feature:  3
Accuracy:  0.9210526315789473

Selected Feature:  4
Accuracy:  0.9210526315789473

Selected Feature:  5
Accuracy:  0.9298245614035088

Selected Feature:  6
Accuracy:  0.9210526315789473

Selected Feature:  7
Accuracy:  0.9210526315789473

Selected Feature:  8
Accuracy:  0.9298245614035088

Selected Feature:  9
Accuracy:  0.9385964912280702

Selected Feature:  10
Accuracy:  0.9473684210526315

Selected Feature:  11
Accuracy:  0.9473684210526315

Selected Feature:  12
Accuracy:  0.9473684210526315

Selected Feature:  13
Accuracy:  0.9385964912280702

Selected Feature:  14
Accuracy:  0.9385964912280702

Selected Feature:  15
Accuracy:  0.9385964912280702

Selected Feature:  16
Accuracy:  0.9473684210526315

Selected Feature:  17
Accuracy:  0.9385964912280702

Selected Feature:  18
Accuracy:  0.9473684210526315

Selected Feature:  19
Accuracy:  0.9473684210526315

Sel